## Mapas de Símbolos

Agora vamos combinar um mapa base e os dados plotados como camadas separadas. Vamos examinar a rede de voos comerciais dos EUA, considerando tanto os aeroportos quanto as rotas de voo. Para isso, precisaremos de três conjuntos de dados.  
Para o nosso mapa base, usaremos um arquivo TopoJSON dos Estados Unidos com resolução de 10m, contendo elementos para `estados` ou `condados`:


In [ ]:
usa = data.us_10m.url
usa

Para os aeroportos, usaremos um conjunto de dados com campos para as coordenadas `longitude` e `latitude` de cada aeroporto, além do código do aeroporto `iata` &mdash; por exemplo, `'SEA'` para o [Aeroporto Internacional de Seattle-Tacoma](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).


In [ ]:
airports = data.airports.url
airports

Por fim, vamos usar um dataset de rotas de voo, que contém campos `origin` e `destination` com os códigos IATA para os aeroportos correspondentes:

In [ ]:
flights = data.flights_airport.url
flights

Vamos começar criando um mapa base usando a projeção `albersUsa` e adicionar uma camada que exibe marcas `circle` para cada aeroporto:

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_São muitos aeroportos! Obviamente, nem todos são grandes hubs._  

Semelhante ao nosso dataset de códigos postais, os dados dos aeroportos incluem pontos que estão fora dos Estados Unidos continentais. Por isso, novamente vemos pontos no canto superior esquerdo. Talvez queiramos filtrar esses pontos, mas para isso precisamos primeiro entender mais sobre eles.  

_Atualize a projeção do mapa acima para `albers` &ndash; contornando o comportamento peculiar do `albersUsa` &ndash; para que as localizações reais desses pontos adicionais sejam reveladas!_  

Agora, em vez de mostrar todos os aeroportos de forma indiferenciada, vamos identificar os grandes hubs considerando o número total de rotas que se originam em cada aeroporto. Vamos usar o dataset `routes` como nossa principal fonte de dados: ele contém uma lista de rotas de voo que podemos agrupar para contar o número de rotas para cada aeroporto `origin`.  

No entanto, o dataset `routes` não inclui as _localizações_ dos aeroportos! Para complementar os dados de `routes` com as localizações, precisamos de uma nova transformação de dados: `lookup`. A transformação `lookup` pega um valor de campo em um dataset principal e usa esse valor como uma _chave_ para procurar informações relacionadas em outra tabela. Neste caso, queremos combinar o código do aeroporto `origin` do nosso dataset `routes` com o campo `iata` do dataset `airports`, e então extrair os campos correspondentes de `latitude` e `longitude`.


In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Quais aeroportos dos EUA têm o maior número de rotas de saída?_  

Agora que podemos ver os aeroportos, talvez queiramos interagir com eles para entender melhor a estrutura da rede de tráfego aéreo. Podemos adicionar uma camada de marca `rule` para representar os caminhos de aeroportos `origin` para aeroportos `destination`, o que requer duas transformações `lookup` para recuperar as coordenadas de cada ponto de extremidade. Além disso, podemos usar uma seleção `single` para filtrar essas rotas, de forma que apenas as rotas originadas no aeroporto atualmente selecionado sejam mostradas.

_Começando a partir do mapa estático acima, você consegue construir uma versão interativa? Fique à vontade para pular o código abaixo e interagir com o mapa interativo primeiro, e depois pense em como construir isso por conta própria!_


In [ ]:
# interactive selection for origin airport
# select nearest airport to mouse cursor
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# shared data reference for lookup transforms
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])
    
alt.layer(
    # base map of the United States
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # route lines from selected origin airport to destination airports
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filter to selected origin only
    ).transform_lookup(
        lookup='origin', from_=foreign # origin lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # dest lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # size airports by number of outgoing routes
    # 1. aggregate flights-airport data set
    # 2. lookup location data from airports data set
    # 3. remove Puerto Rico (PR) and Virgin Islands (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # place smaller circles on top
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

Passe o mouse sobre o mapa para conhecer a rede de voos!